In [ ]:


# Directory containing the files
dir_path = '229r/all1 copy 3'

# Directory containing the files
dat_dir_path = 'data/229r/all1 copy 3'


In [86]:
import pandas as pd
import plotly.graph_objs as go
import plotly.io as pio
file_name = """229r/task_date_{'x': 10, 'y': 10}_{'NOT': 1.0, 'NAND': 1.0, 'AND': 2.0, 'ORN': 2.0, 'OR': 3.0, 'ANDN': 3.0, 'NOR': 4.0, 'XOR': 4.0, 'EQU': 5.0}_03-06_20-38-48.txt"""

def plot_graph(file_name="""229r/task_date_{'x': 10, 'y': 10}_{'NOT': 1.0, 'NAND': 1.0, 'AND': 2.0, 'ORN': 2.0, 'OR': 3.0, 'ANDN': 3.0, 'NOR': 4.0, 'XOR': 4.0, 'EQU': 5.0}_03-06_20-38-48.txt""", title='Average Fitness over Time', width = 800,    height = 600):

    # Read data from file into a list of strings
    with open(file_name, 'r') as f:
        lines = f.readlines()

    # Parse the lines to create a list of dictionaries, where each dictionary contains values for each column
    data = []
    for line in lines:
        if line.startswith('UD:'):
            values = line.strip().split()
            # print(values)

            data.append({
                'UD': int(values[1]),
                'Gen': float(values[3]),
                'Fit': float(values[5]),
                'Orgs': int(values[7])
            })

    # Create dataframe from list of dictionaries
    df = pd.DataFrame(data)
        

    # Define x and y data
    x_data = df.iloc[:, 0] # First column as x-axis
    y_data = df[["Fit"]]
    # y_data = df.iloc[:, 1:] # Every other column as y-axis

    # Create plotly trace for each column
    data = []
    for column in y_data.columns:
        trace = go.Scatter(x=x_data, y=y_data[column], mode='lines', name=column)
        data.append(trace)

    # Define plot layout
    layout = go.Layout(title=title, xaxis=dict(title='# of updates'), yaxis=dict(title='average fitness'), margin=dict(t=40, b=20, l=20, r=20), autosize=False, width=width, height=height)

    # Create plotly figure
    fig = go.Figure(data=data, layout=layout)

    # Display plot
    file_parts = file_name.split('/')
    fig.show()
    pio.write_image(fig, file_parts[0]+'/plots/'+file_parts[1].split('.')[0]+'.png', width=width, height=height, scale=2)


In [87]:
import os
import pandas as pd
import re
# List to store the file names and last lines
file_data = []

# Regular expression pattern
pattern = r"(\d+)_mxc_(\w+)_(\[.*\])_date_(\d{2}-\d{2})_(\d{2}-\d{2}-\d{2})_xydict_values\((\[.*\])\)\.txt"
# Create an empty DataFrame with the required columns
column_names = ['index', 'max_count', 'values', 'experiment_start_time_string', 'xy_values', 'Last_UD', 'Gen', 'Fit', 'Orgs']
df = pd.DataFrame(columns=column_names)

# Create an empty list to store the data
data_list = []

# Loop through all the files in the directory
for filename in os.listdir(dir_path):
    # Check if the file is a text file
    if filename.endswith('.txt'):
        # Open the file and read the last line
        with open(os.path.join(dir_path, filename), 'r') as f:
            # Search for pattern in the log_file_name
            match = re.search(pattern, filename)
            if not match:
                raise Exception("No match found for pattern in file name: {}".format(filename))
            index = int(match.group(1))
            max_count = match.group(2) == 'True'
            values = tuple(int(i) for i in match.group(3).strip('[]').split(', '))
            experiment_start_time_string = match.group(4) + "_" + match.group(5)
            xy_values = tuple(int(i) for i in match.group(6).strip('[]').split(', '))
            
            lines = f.readlines()
            last_line = lines[-1].strip()
            
            with open(os.path.join(dat_dir_path, filename), 'r') as fdat: 
                lines_dat = fdat.readlines()
                last_line_dat = lines_dat[-1].strip()
                numbers = last_line_dat.split()

                dictionary = {
                    "Update": int(numbers[0]),
                    "Not": int(numbers[1]),
                    "Nand": int(numbers[2]),
                    "And": int(numbers[3]),
                    "OrNot": int(numbers[4]),
                    "Or": int(numbers[5]),
                    "AndNot": int(numbers[6]),
                    "Nor": int(numbers[7]),
                    "Xor": int(numbers[8])
                }
                EQU_dict = {
                    "Equals": int(numbers[9])}

            
            if last_line.startswith('UD:'):
                last_line_string = last_line.strip().split()

                data = {
                'max_count': max_count,
                'values': values,
                'last_line_string': last_line_string,
                'experiment_start_time_string': experiment_start_time_string,
                'xy_values': xy_values,
                'Last_UD': int(last_line_string[1]),
                'Gen': float(last_line_string[3]),
                'Fit': float(last_line_string[5]),
                'Orgs': int(last_line_string[7]),
                'index': index,
                'filename': filename,
                }
                data.update(dictionary)
                EQU_dict.update(data)

                # Append the data to the list
                data_list.append(EQU_dict)
            else: 
                raise Exception("Last line does not start with UD: {}".format(filename))

# Create the DataFrame from the data list
df = pd.DataFrame(data_list)

# Print the dataframe
df


,Equals,max_count,values,last_line_string,experiment_start_time_string,xy_values,Last_UD,Gen,Fit,Orgs,...,filename,Update,Not,Nand,And,OrNot,Or,AndNot,Nor,Xor
0,1826,True,"(2, 3, 1, 4, 5)","[UD:, 4400, Gen:, 1017.648, Fit:, 5.304728e+72...",04-13_20-35-15,"(120, 120)",4400,1017.6480,5.304728e+72,14345,...,"0_mxc_True_[2, 3, 1, 4, 5]_date_04-13_20-35-15...",4400,12397,13294,11875,13104,49,542,9074,0
1,3389,True,"(4, 1, 2, 3, 5)","[UD:, 3100, Gen:, 792.0930, Fit:, 9.769858e+44...",04-15_03-32-04,"(120, 120)",3100,792.0930,9.769858e+44,14360,...,"3_mxc_True_[4, 1, 2, 3, 5]_date_04-15_03-32-04...",3100,12099,12937,315,11570,2853,10150,4093,7
2,3641,True,"(1, 2, 3, 4, 6)","[UD:, 10000, Gen:, 2538.028, Fit:, 4.369271e+1...",04-15_05-00-46,"(120, 120)",10000,2538.0280,4.369271e+110,14248,...,"4_mxc_True_[1, 2, 3, 4, 6]_date_04-15_05-00-46...",10000,12988,10536,7502,12010,12113,12095,11874,8
3,0,False,"(1, 2, 3, 4, 6)","[UD:, 10000, Gen:, 1331.495, Fit:, 1.073964e+1...",04-13_15-10-40,"(120, 120)",10000,1331.4950,1.073964e+158,14259,...,"0_mxc_False_[1, 2, 3, 4, 6]_date_04-13_15-10-4...",10000,11353,404,12,10494,11233,121,2247,0
4,0,False,"(3, 1, 2, 4, 5)","[UD:, 2200, Gen:, 1359.863, Fit:, 1.065862e+14...",04-14_13-31-51,"(120, 120)",2200,1359.8630,1.065862e+146,14153,...,"2_mxc_False_[3, 1, 2, 4, 5]_date_04-14_13-31-5...",2200,13518,4609,13,74,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
310,0,False,"(1, 4, 2, 3, 5)","[UD:, 1200, Gen:, 219.3457, Fit:, 3.566222e+09...",04-14_23-43-33,"(120, 120)",1200,219.3457,3.566222e+09,11530,...,"3_mxc_False_[1, 4, 2, 3, 5]_date_04-14_23-43-3...",1200,1506,1214,1912,3145,162,28,2104,0
311,0,False,"(2, 4, 1, 3, 5)","[UD:, 1400, Gen:, 634.9588, Fit:, 2.675431e+65...",04-15_14-12-59,"(120, 120)",1400,634.9588,2.675431e+65,12900,...,"5_mxc_False_[2, 4, 1, 3, 5]_date_04-15_14-12-5...",1400,7798,7949,449,9076,64,8,17,0
312,0,False,"(2, 4, 3, 1, 5)","[UD:, 1300, Gen:, 125.8101, Fit:, 1.004430e+21...",04-15_01-55-27,"(120, 120)",1300,125.8101,1.004430e+211,5908,...,"3_mxc_False_[2, 4, 3, 1, 5]_date_04-15_01-55-2...",1300,103,1962,3,2358,7,0,0,0
313,0,True,"(3, 1, 4, 2, 5)","[UD:, 10000, Gen:, 2230.110, Fit:, 39000.65, O...",04-14_13-33-19,"(120, 120)",10000,2230.1100,3.900065e+04,14347,...,"2_mxc_True_[3, 1, 4, 2, 5]_date_04-14_13-33-19...",10000,13569,12583,10077,11806,13063,11254,12363,0


In [88]:
import pandas as pd
import pickle

# Pickle the dataframe to a file
with open('229r/results/all1_copy_fitness_and_counts_df.pickle', 'wb') as f:
    pickle.dump(df, f)


In [89]:
import pandas as pd
import pickle

# Load the pickled dataframe from the file
with open('229r/results/all1_copy_fitness_and_counts_df.pickle', 'rb') as f:
    df = pickle.load(f)

# Print the dataframe
display(df)


,Equals,max_count,values,last_line_string,experiment_start_time_string,xy_values,Last_UD,Gen,Fit,Orgs,...,filename,Update,Not,Nand,And,OrNot,Or,AndNot,Nor,Xor
0,1826,True,"(2, 3, 1, 4, 5)","[UD:, 4400, Gen:, 1017.648, Fit:, 5.304728e+72...",04-13_20-35-15,"(120, 120)",4400,1017.6480,5.304728e+72,14345,...,"0_mxc_True_[2, 3, 1, 4, 5]_date_04-13_20-35-15...",4400,12397,13294,11875,13104,49,542,9074,0
1,3389,True,"(4, 1, 2, 3, 5)","[UD:, 3100, Gen:, 792.0930, Fit:, 9.769858e+44...",04-15_03-32-04,"(120, 120)",3100,792.0930,9.769858e+44,14360,...,"3_mxc_True_[4, 1, 2, 3, 5]_date_04-15_03-32-04...",3100,12099,12937,315,11570,2853,10150,4093,7
2,3641,True,"(1, 2, 3, 4, 6)","[UD:, 10000, Gen:, 2538.028, Fit:, 4.369271e+1...",04-15_05-00-46,"(120, 120)",10000,2538.0280,4.369271e+110,14248,...,"4_mxc_True_[1, 2, 3, 4, 6]_date_04-15_05-00-46...",10000,12988,10536,7502,12010,12113,12095,11874,8
3,0,False,"(1, 2, 3, 4, 6)","[UD:, 10000, Gen:, 1331.495, Fit:, 1.073964e+1...",04-13_15-10-40,"(120, 120)",10000,1331.4950,1.073964e+158,14259,...,"0_mxc_False_[1, 2, 3, 4, 6]_date_04-13_15-10-4...",10000,11353,404,12,10494,11233,121,2247,0
4,0,False,"(3, 1, 2, 4, 5)","[UD:, 2200, Gen:, 1359.863, Fit:, 1.065862e+14...",04-14_13-31-51,"(120, 120)",2200,1359.8630,1.065862e+146,14153,...,"2_mxc_False_[3, 1, 2, 4, 5]_date_04-14_13-31-5...",2200,13518,4609,13,74,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
310,0,False,"(1, 4, 2, 3, 5)","[UD:, 1200, Gen:, 219.3457, Fit:, 3.566222e+09...",04-14_23-43-33,"(120, 120)",1200,219.3457,3.566222e+09,11530,...,"3_mxc_False_[1, 4, 2, 3, 5]_date_04-14_23-43-3...",1200,1506,1214,1912,3145,162,28,2104,0
311,0,False,"(2, 4, 1, 3, 5)","[UD:, 1400, Gen:, 634.9588, Fit:, 2.675431e+65...",04-15_14-12-59,"(120, 120)",1400,634.9588,2.675431e+65,12900,...,"5_mxc_False_[2, 4, 1, 3, 5]_date_04-15_14-12-5...",1400,7798,7949,449,9076,64,8,17,0
312,0,False,"(2, 4, 3, 1, 5)","[UD:, 1300, Gen:, 125.8101, Fit:, 1.004430e+21...",04-15_01-55-27,"(120, 120)",1300,125.8101,1.004430e+211,5908,...,"3_mxc_False_[2, 4, 3, 1, 5]_date_04-15_01-55-2...",1300,103,1962,3,2358,7,0,0,0
313,0,True,"(3, 1, 4, 2, 5)","[UD:, 10000, Gen:, 2230.110, Fit:, 39000.65, O...",04-14_13-33-19,"(120, 120)",10000,2230.1100,3.900065e+04,14347,...,"2_mxc_True_[3, 1, 4, 2, 5]_date_04-14_13-33-19...",10000,13569,12583,10077,11806,13063,11254,12363,0


In [90]:
df.describe()

,Equals,Last_UD,Gen,Fit,Orgs,index,Update,Not,Nand,And,OrNot,Or,AndNot,Nor,Xor
count,315.000000,315.000000,315.000000,3.150000e+02,315.000000,315.000000,315.000000,315.000000,315.000000,315.000000,315.000000,315.000000,315.000000,315.000000,315.000000
mean,1742.146032,6820.790476,1690.556303,1.412773e+298,13102.711111,2.336508,6820.317460,8483.031746,8077.600000,4130.990476,8144.206349,5423.053968,5650.869841,3912.142857,774.822222
std,3466.971196,3778.272277,1485.635242,inf,2301.177320,1.715729,3778.758081,5030.230517,5294.069233,5303.345458,5144.808247,5458.367291,5594.752676,4998.687704,2589.692444
min,0.000000,500.000000,45.979350,2.596068e-01,3583.000000,0.000000,500.000000,103.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,2500.000000,502.710150,6.658217e+04,13177.000000,1.000000,2500.000000,3163.500000,2161.500000,7.000000,2655.500000,18.500000,15.000000,0.500000,0.000000
50%,0.000000,10000.000000,1727.170000,5.971620e+28,14225.000000,2.000000,10000.000000,10673.000000,10329.000000,205.000000,10759.000000,2993.000000,3258.000000,135.000000,0.000000
75%,887.500000,10000.000000,2338.242500,5.300844e+155,14362.000000,4.000000,10000.000000,13124.500000,13163.000000,10610.500000,12710.000000,11611.500000,11697.500000,9592.000000,1.000000
max,12723.000000,10000.000000,13379.480000,3.391337e+300,14391.000000,5.000000,10000.000000,14061.000000,14174.000000,13331.000000,14077.000000,13722.000000,13855.000000,12857.000000,12445.000000


In [91]:
df.columns

Index(['Equals', 'max_count', 'values', 'last_line_string',
       'experiment_start_time_string', 'xy_values', 'Last_UD', 'Gen', 'Fit',
       'Orgs', 'index', 'filename', 'Update', 'Not', 'Nand', 'And', 'OrNot',
       'Or', 'AndNot', 'Nor', 'Xor'],
      dtype='object')

In [92]:
# Get all the rows with max_count = False and Last_UD = 10000
filtered_df = df[(df['max_count'] == False) & (df['Last_UD'] == 10000)]

In [93]:
filtered_df

,Equals,max_count,values,last_line_string,experiment_start_time_string,xy_values,Last_UD,Gen,Fit,Orgs,...,filename,Update,Not,Nand,And,OrNot,Or,AndNot,Nor,Xor
3,0,False,"(1, 2, 3, 4, 6)","[UD:, 10000, Gen:, 1331.495, Fit:, 1.073964e+1...",04-13_15-10-40,"(120, 120)",10000,1331.4950,1.073964e+158,14259,...,"0_mxc_False_[1, 2, 3, 4, 6]_date_04-13_15-10-4...",10000,11353,404,12,10494,11233,121,2247,0
19,6,False,"(2, 4, 1, 3, 5)","[UD:, 10000, Gen:, 5787.453, Fit:, 1.116889e+1...",04-15_01-33-31,"(120, 120)",10000,5787.4530,1.116889e+165,13590,...,"3_mxc_False_[2, 4, 1, 3, 5]_date_04-15_01-33-3...",10000,6413,8180,9335,10509,27,89,2,0
32,0,False,"(3, 2, 4, 1, 5)","[UD:, 10000, Gen:, 706.1867, Fit:, 5.579337e+1...",04-14_06-14-08,"(120, 120)",10000,706.1867,5.579337e+119,12930,...,"1_mxc_False_[3, 2, 4, 1, 5]_date_04-14_06-14-0...",10000,4741,140,0,1956,3589,10,21,0
37,0,False,"(3, 4, 1, 2, 5)","[UD:, 10000, Gen:, 1314.804, Fit:, 7.214520e+3...",04-14_14-50-55,"(120, 120)",10000,1314.8040,7.214520e+35,9911,...,"2_mxc_False_[3, 4, 1, 2, 5]_date_04-14_14-50-5...",10000,2132,6866,3238,6854,20,11,4,10
44,0,False,"(3, 2, 1, 4, 5)","[UD:, 10000, Gen:, 2510.594, Fit:, 1.596592e+1...",04-15_02-31-58,"(120, 120)",10000,2510.5940,1.596592e+136,14043,...,"3_mxc_False_[3, 2, 1, 4, 5]_date_04-15_02-31-5...",10000,1294,13010,705,12304,2,602,0,0
45,0,False,"(3, 2, 1, 4, 5)","[UD:, 10000, Gen:, 749.8986, Fit:, 4.651467e+1...",04-14_14-11-05,"(120, 120)",10000,749.8986,4.651467e+166,9318,...,"2_mxc_False_[3, 2, 1, 4, 5]_date_04-14_14-11-0...",10000,3391,37,0,4,0,0,0,0
49,0,False,"(4, 2, 3, 1, 5)","[UD:, 10000, Gen:, 11663.70, Fit:, 9.229126e+6...",04-15_04-21-05,"(120, 120)",10000,11663.7000,9.229126e+69,13952,...,"3_mxc_False_[4, 2, 3, 1, 5]_date_04-15_04-21-0...",10000,12549,3076,1,205,3,1,0,0
51,0,False,"(2, 1, 4, 3, 5)","[UD:, 10000, Gen:, 3413.283, Fit:, 1.174972e+4...",04-15_00-34-45,"(120, 120)",10000,3413.2830,1.174972e+45,14069,...,"3_mxc_False_[2, 1, 4, 3, 5]_date_04-15_00-34-4...",10000,13435,13843,15,746,64,3388,12,0
53,0,False,"(1, 3, 4, 2, 5)","[UD:, 10000, Gen:, 3151.796, Fit:, 4.911400e+2...",04-14_23-16-04,"(120, 120)",10000,3151.7960,4.911400e+268,14086,...,"3_mxc_False_[1, 3, 4, 2, 5]_date_04-14_23-16-0...",10000,12936,1100,3,12929,13051,1208,48,0
59,0,False,"(1, 4, 2, 3, 5)","[UD:, 10000, Gen:, 3841.312, Fit:, 3.357075e+9...",04-14_10-35-02,"(120, 120)",10000,3841.3120,3.357075e+94,13780,...,"2_mxc_False_[1, 4, 2, 3, 5]_date_04-14_10-35-0...",10000,9730,238,2,11735,19,10099,0,0


In [94]:
sorted_EQU_df = df.sort_values("Equals", ascending=False)

In [95]:
sorted_EQU_df

,Equals,max_count,values,last_line_string,experiment_start_time_string,xy_values,Last_UD,Gen,Fit,Orgs,...,filename,Update,Not,Nand,And,OrNot,Or,AndNot,Nor,Xor
305,12723,True,"(4, 1, 2, 3, 5)","[UD:, 10000, Gen:, 2169.567, Fit:, 5.271931e+2...",04-14_15-15-07,"(120, 120)",10000,2169.5670,5.271931e+27,14260,...,"2_mxc_True_[4, 1, 2, 3, 5]_date_04-14_15-15-07...",10000,13532,13302,649,13460,13100,5681,11592,12445
95,12049,True,"(3, 1, 2, 4, 5)","[UD:, 7200, Gen:, 1971.225, Fit:, 6.301338e+07...",04-15_01-55-42,"(120, 120)",7200,1971.2250,6.301338e+07,14367,...,"3_mxc_True_[3, 1, 2, 4, 5]_date_04-15_01-55-42...",7200,13194,13522,162,12983,16,13137,12308,1
20,11855,True,"(4, 3, 1, 2, 5)","[UD:, 10000, Gen:, 2338.887, Fit:, 1185905., O...",04-14_00-00-48,"(120, 120)",10000,2338.8870,1.185905e+06,14309,...,"0_mxc_True_[4, 3, 1, 2, 5]_date_04-14_00-00-48...",10000,12938,13563,12421,12757,411,342,13,0
69,11343,True,"(4, 1, 3, 2, 5)","[UD:, 10000, Gen:, 2283.541, Fit:, 2.116421e+1...",04-14_15-42-47,"(120, 120)",10000,2283.5410,2.116421e+14,14387,...,"2_mxc_True_[4, 1, 3, 2, 5]_date_04-14_15-42-47...",10000,13764,12981,11442,12922,11911,346,154,2
136,11307,True,"(3, 1, 2, 4, 5)","[UD:, 4000, Gen:, 1542.077, Fit:, 4.576376e+24...",04-13_21-41-21,"(120, 120)",4000,1542.0770,4.576376e+243,14375,...,"0_mxc_True_[3, 1, 2, 4, 5]_date_04-13_21-41-21...",4000,12621,12923,66,12904,40,381,36,10871
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165,0,True,"(1, 2, 3, 4, 2)","[UD:, 10000, Gen:, 2639.271, Fit:, 14877.09, O...",04-13_16-03-43,"(120, 120)",10000,2639.2710,1.487709e+04,14377,...,"0_mxc_True_[1, 2, 3, 4, 2]_date_04-13_16-03-43...",10000,12937,12935,12470,12633,11669,13434,11356,0
44,0,False,"(3, 2, 1, 4, 5)","[UD:, 10000, Gen:, 2510.594, Fit:, 1.596592e+1...",04-15_02-31-58,"(120, 120)",10000,2510.5940,1.596592e+136,14043,...,"3_mxc_False_[3, 2, 1, 4, 5]_date_04-15_02-31-5...",10000,1294,13010,705,12304,2,602,0,0
167,0,False,"(4, 2, 3, 1, 5)","[UD:, 10000, Gen:, 3499.213, Fit:, 4.977952e+1...",04-14_16-34-43,"(120, 120)",10000,3499.2130,4.977952e+116,14174,...,"2_mxc_False_[4, 2, 3, 1, 5]_date_04-14_16-34-4...",10000,14061,14070,203,2621,10405,46,11,0
168,0,False,"(1, 2, 3, 4, 6)","[UD:, 3100, Gen:, 195.4073, Fit:, 1.604692e+23...",04-14_17-42-12,"(120, 120)",3100,195.4073,1.604692e+233,14379,...,"3_mxc_False_[1, 2, 3, 4, 6]_date_04-14_17-42-1...",3100,914,1901,13,1580,988,1145,1380,0


In [96]:
cleaned_df = sorted_EQU_df[sorted_EQU_df['Last_UD'] == 10000]
display(cleaned_df)

,Equals,max_count,values,last_line_string,experiment_start_time_string,xy_values,Last_UD,Gen,Fit,Orgs,...,filename,Update,Not,Nand,And,OrNot,Or,AndNot,Nor,Xor
305,12723,True,"(4, 1, 2, 3, 5)","[UD:, 10000, Gen:, 2169.567, Fit:, 5.271931e+2...",04-14_15-15-07,"(120, 120)",10000,2169.5670,5.271931e+27,14260,...,"2_mxc_True_[4, 1, 2, 3, 5]_date_04-14_15-15-07...",10000,13532,13302,649,13460,13100,5681,11592,12445
20,11855,True,"(4, 3, 1, 2, 5)","[UD:, 10000, Gen:, 2338.887, Fit:, 1185905., O...",04-14_00-00-48,"(120, 120)",10000,2338.8870,1.185905e+06,14309,...,"0_mxc_True_[4, 3, 1, 2, 5]_date_04-14_00-00-48...",10000,12938,13563,12421,12757,411,342,13,0
69,11343,True,"(4, 1, 3, 2, 5)","[UD:, 10000, Gen:, 2283.541, Fit:, 2.116421e+1...",04-14_15-42-47,"(120, 120)",10000,2283.5410,2.116421e+14,14387,...,"2_mxc_True_[4, 1, 3, 2, 5]_date_04-14_15-42-47...",10000,13764,12981,11442,12922,11911,346,154,2
202,11215,True,"(1, 3, 4, 2, 5)","[UD:, 10000, Gen:, 1629.322, Fit:, 2.024988e+0...",04-15_07-26-45,"(120, 120)",10000,1629.3220,2.024988e+09,14359,...,"4_mxc_True_[1, 3, 4, 2, 5]_date_04-15_07-26-45...",10000,12706,4405,12312,12502,11763,12499,20,1
35,11050,True,"(1, 2, 3, 4, 4)","[UD:, 10000, Gen:, 2484.113, Fit:, 3.903534e+0...",04-14_17-44-27,"(120, 120)",10000,2484.1130,3.903534e+09,14350,...,"3_mxc_True_[1, 2, 3, 4, 4]_date_04-14_17-44-27...",10000,13089,13119,263,12284,12211,11323,10956,38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45,0,False,"(3, 2, 1, 4, 5)","[UD:, 10000, Gen:, 749.8986, Fit:, 4.651467e+1...",04-14_14-11-05,"(120, 120)",10000,749.8986,4.651467e+166,9318,...,"2_mxc_False_[3, 2, 1, 4, 5]_date_04-14_14-11-0...",10000,3391,37,0,4,0,0,0,0
164,0,True,"(2, 4, 3, 1, 5)","[UD:, 10000, Gen:, 2037.453, Fit:, 97287.39, O...",04-14_05-04-15,"(120, 120)",10000,2037.4530,9.728739e+04,14379,...,"1_mxc_True_[2, 4, 3, 1, 5]_date_04-14_05-04-15...",10000,13531,13261,11306,12894,12395,12767,11,9983
165,0,True,"(1, 2, 3, 4, 2)","[UD:, 10000, Gen:, 2639.271, Fit:, 14877.09, O...",04-13_16-03-43,"(120, 120)",10000,2639.2710,1.487709e+04,14377,...,"0_mxc_True_[1, 2, 3, 4, 2]_date_04-13_16-03-43...",10000,12937,12935,12470,12633,11669,13434,11356,0
44,0,False,"(3, 2, 1, 4, 5)","[UD:, 10000, Gen:, 2510.594, Fit:, 1.596592e+1...",04-15_02-31-58,"(120, 120)",10000,2510.5940,1.596592e+136,14043,...,"3_mxc_False_[3, 2, 1, 4, 5]_date_04-15_02-31-5...",10000,1294,13010,705,12304,2,602,0,0


In [97]:
# cleaned_df[cleaned_df["max_count"]].describe().to_latex(index=False)
d1 = cleaned_df[cleaned_df["max_count"]]
d1 = d1[["Equals", "Fit"]]
d1 = d1.describe()
d1 = d1.drop(["count", "std"])
print(d1.to_latex(index=True))

\begin{tabular}{lrr}
\toprule
{} &        Equals &            Fit \\
\midrule
mean &   2793.289256 &  3.280936e+296 \\
min  &      0.000000 &   6.751146e+01 \\
25\%  &      0.000000 &   1.508415e+04 \\
50\%  &      0.000000 &   6.047860e+04 \\
75\%  &   5402.000000 &   2.889303e+07 \\
max  &  12723.000000 &  3.969932e+298 \\
\bottomrule
\end{tabular}



/var/folders/nk/6df909tn2f726qntp17fzj600000gn/T/ipykernel_40292/2916242337.py:6: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(d1.to_latex(index=True))


In [98]:
# display(cleaned_df[~cleaned_df["max_count"]].describe())
# cleaned_df[cleaned_df["max_count"]].describe().to_latex(index=False)
d1 = cleaned_df[~cleaned_df["max_count"]]
d1 = d1[["Equals", "Fit", "filename"]]
d1 = d1.describe()
d1 = d1.drop(["count", "std"])
print(d1.to_latex(index=True))

\begin{tabular}{lrr}
\toprule
{} &    Equals &            Fit \\
\midrule
mean &  0.238095 &  3.219152e+292 \\
min  &  0.000000 &   8.355300e+05 \\
25\%  &  0.000000 &   2.285031e+72 \\
50\%  &  0.000000 &  2.948822e+119 \\
75\%  &  0.000000 &  8.376668e+164 \\
max  &  6.000000 &  1.352044e+294 \\
\bottomrule
\end{tabular}



/var/folders/nk/6df909tn2f726qntp17fzj600000gn/T/ipykernel_40292/652201068.py:7: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(d1.to_latex(index=True))


In [99]:
cleaned_df = cleaned_df[["values", "max_count", "Equals", "Fit", "index", "filename"]]

In [100]:
diff_vals_df = cleaned_df.sort_values("values")

In [101]:
print(diff_vals_df.to_latex(index=False))

\begin{tabular}{llrrrl}
\toprule
         values &  max\_count &  Equals &           Fit &  index &                                           filename \\
\midrule
(1, 2, 3, 4, 1) &       True &       0 &  1.572282e+04 &      5 & 5\_mxc\_True\_[1, 2, 3, 4, 1]\_date\_04-15\_10-38-25\_... \\
(1, 2, 3, 4, 1) &       True &       0 &  9.568449e+03 &      3 & 3\_mxc\_True\_[1, 2, 3, 4, 1]\_date\_04-14\_21-09-37\_... \\
(1, 2, 3, 4, 1) &       True &       0 &  3.881085e+03 &      1 & 1\_mxc\_True\_[1, 2, 3, 4, 1]\_date\_04-14\_01-56-53\_... \\
(1, 2, 3, 4, 1) &       True &       0 &  3.653363e+03 &      2 & 2\_mxc\_True\_[1, 2, 3, 4, 1]\_date\_04-14\_09-01-14\_... \\
(1, 2, 3, 4, 1) &       True &       0 &  4.515025e+03 &      4 & 4\_mxc\_True\_[1, 2, 3, 4, 1]\_date\_04-15\_06-26-03\_... \\
(1, 2, 3, 4, 1) &      False &       0 &  9.591164e+12 &      5 & 5\_mxc\_False\_[1, 2, 3, 4, 1]\_date\_04-15\_10-56-28... \\
(1, 2, 3, 4, 1) &       True &      19 &  1.712673e+04 &      0 & 0\_mxc\_Tr

/var/folders/nk/6df909tn2f726qntp17fzj600000gn/T/ipykernel_40292/128294827.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(diff_vals_df.to_latex(index=False))


In [102]:
display(diff_vals_df)

,values,max_count,Equals,Fit,index,filename
75,"(1, 2, 3, 4, 1)",True,0,15722.820,5,"5_mxc_True_[1, 2, 3, 4, 1]_date_04-15_10-38-25..."
76,"(1, 2, 3, 4, 1)",True,0,9568.449,3,"3_mxc_True_[1, 2, 3, 4, 1]_date_04-14_21-09-37..."
39,"(1, 2, 3, 4, 1)",True,0,3881.085,1,"1_mxc_True_[1, 2, 3, 4, 1]_date_04-14_01-56-53..."
306,"(1, 2, 3, 4, 1)",True,0,3653.363,2,"2_mxc_True_[1, 2, 3, 4, 1]_date_04-14_09-01-14..."
222,"(1, 2, 3, 4, 1)",True,0,4515.025,4,"4_mxc_True_[1, 2, 3, 4, 1]_date_04-15_06-26-03..."
...,...,...,...,...,...,...
181,"(4, 3, 1, 2, 5)",True,0,45712.830,3,"3_mxc_True_[4, 3, 1, 2, 5]_date_04-15_04-29-04..."
184,"(4, 3, 2, 1, 5)",True,2,125423.800,0,"0_mxc_True_[4, 3, 2, 1, 5]_date_04-14_00-21-26..."
268,"(4, 3, 2, 1, 5)",True,0,95153.350,2,"2_mxc_True_[4, 3, 2, 1, 5]_date_04-14_17-11-02..."
177,"(4, 3, 2, 1, 5)",True,0,102221.400,1,"1_mxc_True_[4, 3, 2, 1, 5]_date_04-14_07-38-08..."


In [103]:
fdf = diff_vals_df 

In [104]:
gdf = diff_vals_df.groupby(["values", "max_count"]).mean()
gdf

Equals            Fit     index
values          max_count                                       
(1, 2, 3, 4, 1) False          0.000000   9.591164e+12  5.000000
                True           3.166667   9.077912e+03  2.500000
(1, 2, 3, 4, 2) True        1468.000000   1.565792e+72  2.500000
(1, 2, 3, 4, 3) False          0.000000  1.006586e+282  2.000000
                True        4714.833333  6.616553e+297  2.500000
(1, 2, 3, 4, 4) False          0.000000  6.942963e+198  2.666667
                True        3964.833333   1.838852e+33  2.500000
(1, 2, 3, 4, 5) False          0.000000  7.957690e+117  2.500000
                True        2655.600000  2.003442e+284  2.800000
(1, 2, 3, 4, 6) False          0.333333  6.739933e+270  1.666667
                True        5888.000000  1.456424e+110  2.333333
(1, 2, 4, 3, 5) False          1.000000   3.635732e+42  2.000000
                True        2190.400000  1.367504e+215  2.200000
(1, 3, 2, 4, 5) True           0.000000   7.435400e+03  3.250000
(1, 3, 4, 2, 5) False          0.000000  2.455700e+268  1.500000
                True        3220.600000   4.050661e+08  2.400000
(1, 4, 2, 3, 5) False          0.000000  8.129990e+165  3.000000
                True        3002.500000   4.848623e+26  2.000000
(1, 4, 3, 2, 5) False          0.000000  7.333110e+129  2.000000
                True        3367.800000  4.798266e+217  3.000000
(2, 1, 3, 4, 5) False          0.000000  7.043458e+116  0.000000
                True        3197.000000   9.339148e+77  2.400000
(2, 1, 4, 3, 5) False          0.000000   1.174972e+45  3.000000
                True        4083.500000   1.038945e+33  1.500000
(2, 3, 1, 4, 5) False          0.500000  4.846086e+180  1.500000
                True        2671.250000   1.491151e+55  2.750000
(2, 3, 4, 1, 5) False          0.000000   9.685311e+40  2.000000
                True           0.000000   4.011342e+04  2.200000
(2, 4, 1, 3, 5) False          3.000000  5.584445e+164  2.500000
                True        4473.666667   2.050538e+13  1.333333
(2, 4, 3, 1, 5) False          0.000000  1.352044e+294  5.000000
                True        1815.400000  5.637682e+190  2.200000
(3, 1, 2, 4, 5) False          0.000000   4.261628e+21  1.000000
                True        8379.000000   8.500257e+10  5.000000
(3, 1, 4, 2, 5) True        2586.800000   1.159795e+58  2.200000
(3, 2, 1, 4, 5) False          0.000000  1.550489e+166  3.333333
                True        5958.000000   4.814338e+08  2.666667
(3, 2, 4, 1, 5) False          0.000000  5.579337e+119  1.000000
                True        5920.000000  3.616482e+170  2.200000
(3, 4, 1, 2, 5) False          0.000000   7.214520e+35  2.000000
                True           0.000000   1.485955e+04  1.500000
(3, 4, 2, 1, 5) True           0.000000   6.742804e+04  2.500000
(4, 1, 2, 3, 5) False          0.000000   8.878220e+92  3.500000
                True       12723.000000   5.271931e+27  2.000000
(4, 1, 3, 2, 5) True        2835.750000   5.291053e+13  1.500000
(4, 2, 1, 3, 5) True        2273.500000   5.065357e+35  2.500000
(4, 2, 3, 1, 5) False          0.000000  2.488976e+116  2.500000
                True        2732.750000   8.469035e+04  2.500000
(4, 3, 1, 2, 5) False          0.000000  4.589406e+170  1.000000
                True        3951.666667   4.215937e+05  1.666667
(4, 3, 2, 1, 5) True           0.500000   9.710321e+04  1.500000

In [105]:
gdf = gdf.reset_index().sort_values("Equals")

In [106]:
gdf[gdf["max_count"] == False]

,values,max_count,Equals,Fit,index
0,"(1, 2, 3, 4, 1)",False,0.000000,9.591164e+12,5.000000
26,"(2, 3, 4, 1, 5)",False,0.000000,9.685311e+40,2.000000
20,"(2, 1, 3, 4, 5)",False,0.000000,7.043458e+116,0.000000
18,"(1, 4, 3, 2, 5)",False,0.000000,7.333110e+129,2.000000
30,"(2, 4, 3, 1, 5)",False,0.000000,1.352044e+294,5.000000
16,"(1, 4, 2, 3, 5)",False,0.000000,8.129990e+165,3.000000
32,"(3, 1, 2, 4, 5)",False,0.000000,4.261628e+21,1.000000
14,"(1, 3, 4, 2, 5)",False,0.000000,2.455700e+268,1.500000
35,"(3, 2, 1, 4, 5)",False,0.000000,1.550489e+166,3.333333
37,"(3, 2, 4, 1, 5)",False,0.000000,5.579337e+119,1.000000


In [107]:
gdf[gdf["max_count"] == True]

,values,max_count,Equals,Fit,index
27,"(2, 3, 4, 1, 5)",True,0.000000,4.011342e+04,2.200000
13,"(1, 3, 2, 4, 5)",True,0.000000,7.435400e+03,3.250000
40,"(3, 4, 1, 2, 5)",True,0.000000,1.485955e+04,1.500000
41,"(3, 4, 2, 1, 5)",True,0.000000,6.742804e+04,2.500000
50,"(4, 3, 2, 1, 5)",True,0.500000,9.710321e+04,1.500000
1,"(1, 2, 3, 4, 1)",True,3.166667,9.077912e+03,2.500000
2,"(1, 2, 3, 4, 2)",True,1468.000000,1.565792e+72,2.500000
31,"(2, 4, 3, 1, 5)",True,1815.400000,5.637682e+190,2.200000
12,"(1, 2, 4, 3, 5)",True,2190.400000,1.367504e+215,2.200000
45,"(4, 2, 1, 3, 5)",True,2273.500000,5.065357e+35,2.500000


In [108]:
gdf[(gdf["max_count"] == True) & (gdf['values'].apply(lambda tup: tup[0] == 1))]

,values,max_count,Equals,Fit,index
13,"(1, 3, 2, 4, 5)",True,0.000000,7.435400e+03,3.250000
1,"(1, 2, 3, 4, 1)",True,3.166667,9.077912e+03,2.500000
2,"(1, 2, 3, 4, 2)",True,1468.000000,1.565792e+72,2.500000
12,"(1, 2, 4, 3, 5)",True,2190.400000,1.367504e+215,2.200000
8,"(1, 2, 3, 4, 5)",True,2655.600000,2.003442e+284,2.800000
17,"(1, 4, 2, 3, 5)",True,3002.500000,4.848623e+26,2.000000
15,"(1, 3, 4, 2, 5)",True,3220.600000,4.050661e+08,2.400000
19,"(1, 4, 3, 2, 5)",True,3367.800000,4.798266e+217,3.000000
6,"(1, 2, 3, 4, 4)",True,3964.833333,1.838852e+33,2.500000
4,"(1, 2, 3, 4, 3)",True,4714.833333,6.616553e+297,2.500000


In [109]:
gdf[(gdf["max_count"] == True) & (gdf['values'].apply(lambda tup: tup[0] == 4))]

,values,max_count,Equals,Fit,index
50,"(4, 3, 2, 1, 5)",True,0.500000,9.710321e+04,1.500000
45,"(4, 2, 1, 3, 5)",True,2273.500000,5.065357e+35,2.500000
47,"(4, 2, 3, 1, 5)",True,2732.750000,8.469035e+04,2.500000
44,"(4, 1, 3, 2, 5)",True,2835.750000,5.291053e+13,1.500000
49,"(4, 3, 1, 2, 5)",True,3951.666667,4.215937e+05,1.666667
43,"(4, 1, 2, 3, 5)",True,12723.000000,5.271931e+27,2.000000


In [110]:
fdf[fdf["values"] == (4, 1, 2, 3, 5)]

,values,max_count,Equals,Fit,index,filename
305,"(4, 1, 2, 3, 5)",True,12723,5.271931e+27,2,"2_mxc_True_[4, 1, 2, 3, 5]_date_04-14_15-15-07..."
124,"(4, 1, 2, 3, 5)",False,0,1.775644e+93,2,"2_mxc_False_[4, 1, 2, 3, 5]_date_04-14_15-33-4..."
123,"(4, 1, 2, 3, 5)",False,0,3.594581e+49,5,"5_mxc_False_[4, 1, 2, 3, 5]_date_04-15_16-30-4..."


In [115]:
fname = fdf[fdf["values"] == (4, 1, 2, 3, 5)]["filename"].iloc[0]

In [117]:
plot_graph(file_name=dir_path + "/" + fname)